In [1]:
# Import Dependencies
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd
import json

In [2]:
# Web Scraping Function That Returns Desired Web Links For Each Team (<span>)
# [Function] Params: <String> Link Description as Shown on Website
          # Returns: <Dictionary> Team Name: Description Link
        
def espn_mlb_team_links(link_desc):
    _base_url = 'http://www.espn.com'
    _htmldoc = req.get(_base_url + '/mlb/teams').text
    _soup = bs(_htmldoc, 'html.parser')
    _span_tags = _soup.find_all('span', class_='TeamLinks__Link n9 nowrap')

    _team_urls = {}
    for _span in _span_tags:
        _link = _span.a['href'].strip()
        _info = _link.split('/')
        
        if f'{link_desc.strip()}' in _info:
            _team = _info[-1].strip().title()
            _team_url = _base_url + _link
            _team_urls[_team] = _team_url
            print(_team, '\n', _team_url)
    return _team_urls

In [3]:
# Web Scraping Function That Returns Basic Player Info & Links to Personal Page on ESPN (<span>)
# [Function] Params: <Dictionary> Team Roster
                    #Key: Team Name
                    #Value: Link to Team Roster
          # Returns: <Dictionary> Player Gen Info & Personal Page Links(ESPN)
        
def team_player_links(roster):
    _player_links = {}
    for _team, _url in roster.items():
        _htmldoc = req.get(_url).text
        _soup = bs(_htmldoc, 'html.parser')
        _span_tags = _soup.find_all('span', class_='Table2__td--headshot')
        
        for _span in _span_tags:
            try:
                _link = _span.a['href']
                _info = _link.split('/')
            
                if 'player' in _info:
                    _pID = _info[-1]
                    _pName = _span.a.find('figure')['title'].strip().title()
                    _t = _url.split('/').index('name') + 1
                    _tID = _url.split('/')[_t].upper()
                    
                    if _pID not in _player_links:
                        _player_links[_pID] = {
                            'id': _pID,
                            'name': _pName,
                            'team': _team,
                            'tid': _tID,
                            'link': _link
                        }
                        print(f'ID: {_pID}, Name: {_pName}, Team: {_team}\n\tLink: {_link}')
 
            except TypeError:
                print('\n<Skip> No Player Link Found\n')
            
    return _player_links

In [4]:
# Web Scraping Function That Extracts Personal Player Information From ESPN (<li>)

def get_player_info(p_link):
    _htmldoc = req.get(p_link).text
    _soup = bs(_htmldoc, 'html.parser')
    _li_tags = _soup.find_all('li')
    try:
        _num = _li_tags[0].text.split(' ')[0][1:]
        _pos = _li_tags[0].text.split(' ')[1]
        _thr = _li_tags[1].text.split(',')[0][-1]
        _bat = _li_tags[1].text.split(',')[1][-1]
        _team = _li_tags[2].text
        _bday = _li_tags[3].text[10:].split('(')[0].strip()
        _age = _li_tags[3].text[10:].split('(')[1].split(':')[1][:-1].strip()
        _city = _li_tags[4].text[10:].split(',')[0].strip()
        _exp = _li_tags[5].text[10:]
        _col = _li_tags[6].text[7:]
        _hght = _li_tags[7].text.split(',')[0][5:].strip()
        _wght = _li_tags[7].text.split(',')[1].strip()
        
    except IndexError:
        return {
            'pid': None,
            'fname': None,
            'lname': None,
            'age': 0,
            'height': None,
            'weight': None,
            'birthday': None,
            'city': None,
            'number': -1,
            'position': None,
            'exp': None,
            'team': None,
            'tid': None,
            'throw': None,
            'bat': None,
            'college': None,
        }
    return {
        'pid': None,
        'fname': None,
        'lname': None,
        'age': int(_age),
        'height': _hght,
        'weight': _wght,
        'birthday': _bday,
        'city': _city,
        'number': int(_num),
        'position': _pos,
        'exp': _exp,
        'team': _team,
        'tid': None,
        'throw': _thr,
        'bat': _bat,
        'college': _col,
    }

In [5]:
def get_player_season_stats(p_name, p_link):
    _pName = p_name.replace(' ', '-').lower()
    _modLink = p_link.split('/')
    if 'player' in _modLink:
        _insIndex = _modLink.index('player') + 1
        _modLink.insert(_insIndex, 'stats')
        _modLink.append(_pName)
        _statsLink = '/'.join(modLink)
        
        pStats = pd.read_html(_statsLink)
        _sumStats = pStats[3]
        _colHead = _sumStats.iloc[1]
        _sumStats.columns = _colHead
        
        
        
    
    

In [6]:
# Return List of All Team Roster Links
rosters = espn_mlb_team_links('roster')

Chicago-White-Sox 
 http://www.espn.com/mlb/team/roster/_/name/chw/chicago-white-sox
Cleveland-Indians 
 http://www.espn.com/mlb/team/roster/_/name/cle/cleveland-indians
Detroit-Tigers 
 http://www.espn.com/mlb/team/roster/_/name/det/detroit-tigers
Kansas-City-Royals 
 http://www.espn.com/mlb/team/roster/_/name/kc/kansas-city-royals
Minnesota-Twins 
 http://www.espn.com/mlb/team/roster/_/name/min/minnesota-twins
Baltimore-Orioles 
 http://www.espn.com/mlb/team/roster/_/name/bal/baltimore-orioles
Boston-Red-Sox 
 http://www.espn.com/mlb/team/roster/_/name/bos/boston-red-sox
New-York-Yankees 
 http://www.espn.com/mlb/team/roster/_/name/nyy/new-york-yankees
Tampa-Bay-Rays 
 http://www.espn.com/mlb/team/roster/_/name/tb/tampa-bay-rays
Toronto-Blue-Jays 
 http://www.espn.com/mlb/team/roster/_/name/tor/toronto-blue-jays
Houston-Astros 
 http://www.espn.com/mlb/team/roster/_/name/hou/houston-astros
Los-Angeles-Angels 
 http://www.espn.com/mlb/team/roster/_/name/laa/los-angeles-angels
Oakland-

In [7]:
# Return Dictionary Containing Player Gen Info & Link to Personal Page (ESPN)
players = team_player_links(rosters)


<Skip> No Player Link Found

ID: 36131, Name: Aaron Bummer, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/36131
ID: 31255, Name: Alex Colome, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/31255
ID: 33617, Name: Odrisamer Despaigne, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/33617
ID: 34240, Name: Jace Fry, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/34240
ID: 34913, Name: Carson Fulmer, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/34913
ID: 32697, Name: Lucas Giolito, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/32697
ID: 32029, Name: Kelvin Herrera, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/32029
ID: 33860, Name: Reynaldo Lopez, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/33860
ID: 32100, Name: Evan Marshall, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/32100
ID: 34845, Name: Juan Min


<Skip> No Player Link Found

ID: 28668, Name: Homer Bailey, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/28668
ID: 32156, Name: Scott Barlow, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/32156
ID: 31495, Name: Brad Boxberger, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/31495
ID: 32178, Name: Jake Diekman, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/32178
ID: 31114, Name: Danny Duffy, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/31114
ID: 32671, Name: Brian Flynn, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/32671
ID: 36056, Name: Jakob Junis, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/36056
ID: 35292, Name: Brad Keller, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/35292
ID: 28864, Name: Ian Kennedy, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/28864
ID: 33833, Name: Jorge Lopez


<Skip> No Player Link Found

ID: 32147, Name: Matt Barnes, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/32147
ID: 30189, Name: Ryan Brasier, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/30189
ID: 36631, Name: Colten Brewer, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/36631
ID: 34400, Name: Brian Johnson, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/34400
ID: 29966, Name: Rick Porcello, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/29966
ID: 28958, Name: David Price, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/28958
ID: 32675, Name: Eduardo Rodriguez, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/32675
ID: 30948, Name: Chris Sale, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/30948
ID: 40747, Name: Mike Shawaryn, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/40747
ID: 34719, Name: Josh Smith, Team: Boston-Red-Sox
	Link:


<Skip> No Player Link Found

ID: 36887, Name: Rogelio Armenteros, Team: Houston-Astros
	Link: http://www.espn.com/mlb/player/_/id/36887
ID: 32081, Name: Gerrit Cole, Team: Houston-Astros
	Link: http://www.espn.com/mlb/player/_/id/32081
ID: 32511, Name: Chris Devenski, Team: Houston-Astros
	Link: http://www.espn.com/mlb/player/_/id/32511
ID: 32566, Name: Will Harris, Team: Houston-Astros
	Link: http://www.espn.com/mlb/player/_/id/32566
ID: 41128, Name: Josh James, Team: Houston-Astros
	Link: http://www.espn.com/mlb/player/_/id/41128
ID: 31094, Name: Wade Miley, Team: Houston-Astros
	Link: http://www.espn.com/mlb/player/_/id/31094
ID: 32693, Name: Roberto Osuna, Team: Houston-Astros
	Link: http://www.espn.com/mlb/player/_/id/32693
ID: 31591, Name: Brad Peacock, Team: Houston-Astros
	Link: http://www.espn.com/mlb/player/_/id/31591
ID: 33072, Name: Ryan Pressly, Team: Houston-Astros
	Link: http://www.espn.com/mlb/player/_/id/33072
ID: 30089, Name: Hector Rondon, Team: Houston-Astros
	Link


<Skip> No Player Link Found

ID: 34264, Name: Kyle Bird, Team: Texas-Rangers
	Link: http://www.espn.com/mlb/player/_/id/34264
ID: 29220, Name: Jesse Chavez, Team: Texas-Rangers
	Link: http://www.espn.com/mlb/player/_/id/29220
ID: 42180, Name: Peter Fairbanks, Team: Texas-Rangers
	Link: http://www.espn.com/mlb/player/_/id/42180
ID: 35281, Name: Ariel Jurado, Team: Texas-Rangers
	Link: http://www.espn.com/mlb/player/_/id/35281
ID: 30258, Name: Shawn Kelley, Team: Texas-Rangers
	Link: http://www.espn.com/mlb/player/_/id/30258
ID: 33926, Name: Jose Leclerc, Team: Texas-Rangers
	Link: http://www.espn.com/mlb/player/_/id/33926
ID: 30820, Name: Lance Lynn, Team: Texas-Rangers
	Link: http://www.espn.com/mlb/player/_/id/30820
ID: 39815, Name: Brett Martin, Team: Texas-Rangers
	Link: http://www.espn.com/mlb/player/_/id/39815
ID: 32903, Name: Chris Martin, Team: Texas-Rangers
	Link: http://www.espn.com/mlb/player/_/id/32903
ID: 30738, Name: Shelby Miller, Team: Texas-Rangers
	Link: http://www.es


<Skip> No Player Link Found

ID: 31003, Name: Chris Archer, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/31003
ID: 33870, Name: Steven Brault, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/33870
ID: 32171, Name: Kyle Crick, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/32171
ID: 33235, Name: Michael Feliz, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/33235
ID: 40865, Name: Geoff Hartlieb, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/40865
ID: 32827, Name: Clay Holmes, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/32827
ID: 6211, Name: Francisco Liriano, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/6211
ID: 34848, Name: Joe Musgrove, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/34848
ID: 35243, Name: Richard Rodriguez, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/35243
ID: 32787, Name:


<Skip> No Player Link Found

ID: 35394, Name: Edwin Diaz, Team: New-York-Mets
	Link: http://www.espn.com/mlb/player/_/id/35394
ID: 36067, Name: Chris Flexen, Team: New-York-Mets
	Link: http://www.espn.com/mlb/player/_/id/36067
ID: 31075, Name: Wilmer Font, Team: New-York-Mets
	Link: http://www.espn.com/mlb/player/_/id/31075
ID: 34872, Name: Robert Gsellman, Team: New-York-Mets
	Link: http://www.espn.com/mlb/player/_/id/34872
ID: 33293, Name: Walker Lockett, Team: New-York-Mets
	Link: http://www.espn.com/mlb/player/_/id/33293
ID: 34873, Name: Seth Lugo, Team: New-York-Mets
	Link: http://www.espn.com/mlb/player/_/id/34873
ID: 33106, Name: Steven Matz, Team: New-York-Mets
	Link: http://www.espn.com/mlb/player/_/id/33106
ID: 39707, Name: Stephen Nogosek, Team: New-York-Mets
	Link: http://www.espn.com/mlb/player/_/id/39707
ID: 34902, Name: Brooks Pounders, Team: New-York-Mets
	Link: http://www.espn.com/mlb/player/_/id/34902
ID: 6352, Name: Jason Vargas, Team: New-York-Mets
	Link: http://ww


<Skip> No Player Link Found

ID: 32549, Name: Matt Andriese, Team: Arizona-Diamondbacks
	Link: http://www.espn.com/mlb/player/_/id/32549
ID: 32518, Name: Archie Bradley, Team: Arizona-Diamondbacks
	Link: http://www.espn.com/mlb/player/_/id/32518
ID: 32810, Name: Andrew Chafin, Team: Arizona-Diamondbacks
	Link: http://www.espn.com/mlb/player/_/id/32810
ID: 35277, Name: Taylor Clarke, Team: Arizona-Diamondbacks
	Link: http://www.espn.com/mlb/player/_/id/35277
ID: 35681, Name: Stefan Crichton, Team: Arizona-Diamondbacks
	Link: http://www.espn.com/mlb/player/_/id/35681
ID: 34824, Name: Zack Godley, Team: Arizona-Diamondbacks
	Link: http://www.espn.com/mlb/player/_/id/34824
ID: 5883, Name: Zack Greinke, Team: Arizona-Diamondbacks
	Link: http://www.espn.com/mlb/player/_/id/5883
ID: 39842, Name: Yoshihisa Hirano, Team: Arizona-Diamondbacks
	Link: http://www.espn.com/mlb/player/_/id/39842
ID: 30945, Name: Greg Holland, Team: Arizona-Diamondbacks
	Link: http://www.espn.com/mlb/player/_/id/3094


<Skip> No Player Link Found

ID: 35024, Name: Logan Allen, Team: San-Diego-Padres
	Link: http://www.espn.com/mlb/player/_/id/35024
ID: 39915, Name: Eric Lauer, Team: San-Diego-Padres
	Link: http://www.espn.com/mlb/player/_/id/39915
ID: 39876, Name: Joey Lucchesi, Team: San-Diego-Padres
	Link: http://www.espn.com/mlb/player/_/id/39876
ID: 36161, Name: Phil Maton, Team: San-Diego-Padres
	Link: http://www.espn.com/mlb/player/_/id/36161
ID: 35970, Name: Luis Perdomo, Team: San-Diego-Padres
	Link: http://www.espn.com/mlb/player/_/id/35970
ID: 39875, Name: Cal Quantrill, Team: San-Diego-Padres
	Link: http://www.espn.com/mlb/player/_/id/39875
ID: 35765, Name: Gerardo Reyes, Team: San-Diego-Padres
	Link: http://www.espn.com/mlb/player/_/id/35765
ID: 30361, Name: Craig Stammen, Team: San-Diego-Padres
	Link: http://www.espn.com/mlb/player/_/id/30361
ID: 34862, Name: Matt Strahm, Team: San-Diego-Padres
	Link: http://www.espn.com/mlb/player/_/id/34862
ID: 36158, Name: Brad Wieck, Team: San-Diego-

In [8]:
# Iterate Through Each Player's Personal Page and Retrieve Detailed Player Info
mlb_player_data = []
for playerID, playerInfo in players.items():
    _pData = get_player_info(playerInfo['link'])
    _pData['pid'] = playerID
    _pData['fname'] = playerInfo['name'].split(' ')[0].strip()
    _pData['lname'] = playerInfo['name'].split(' ')[1].strip()
    _pData['tid'] = playerInfo['tid']
    mlb_player_data.append(_pData)
    print(json.dumps(_pData, indent=4, sort_keys=False), '\n')

{
    "pid": "36131",
    "fname": "Aaron",
    "lname": "Bummer",
    "age": 25,
    "height": "6-3",
    "weight": "200 lbs.",
    "birthday": "September 21, 1993",
    "city": "Peoria",
    "number": 39,
    "position": "RP",
    "exp": "2 years",
    "team": "Chicago White Sox",
    "tid": "CHW",
    "throw": "L",
    "bat": "L",
    "college": "Nebraska"
} 

{
    "pid": "31255",
    "fname": "Alex",
    "lname": "Colome",
    "age": 30,
    "height": "6-1",
    "weight": "220 lbs.",
    "birthday": "December 31, 1988",
    "city": "Santo Domingo",
    "number": 48,
    "position": "RP",
    "exp": "6 years",
    "team": "Chicago White Sox",
    "tid": "CHW",
    "throw": "R",
    "bat": "R",
    "college": "None"
} 

{
    "pid": "33617",
    "fname": "Odrisamer",
    "lname": "Despaigne",
    "age": 32,
    "height": "6-0",
    "weight": "200 lbs.",
    "birthday": "April 4, 1987",
    "city": "Havana",
    "number": 25,
    "position": "SP",
    "exp": "5 years",
    "team": "C

{
    "pid": "33867",
    "fname": "Eloy",
    "lname": "Jimenez",
    "age": 0,
    "height": null,
    "weight": null,
    "birthday": null,
    "city": null,
    "number": -1,
    "position": null,
    "exp": null,
    "team": null,
    "tid": "CHW",
    "throw": null,
    "bat": null,
    "college": null
} 

{
    "pid": "32866",
    "fname": "Charlie",
    "lname": "Tilson",
    "age": 26,
    "height": "5-11",
    "weight": "195 lbs.",
    "birthday": "December 2, 1992",
    "city": "Wilmette",
    "number": 22,
    "position": "CF",
    "exp": "2 years",
    "team": "Chicago White Sox",
    "tid": "CHW",
    "throw": "L",
    "bat": "L",
    "college": "None"
} 

{
    "pid": "32014",
    "fname": "Trevor",
    "lname": "Bauer",
    "age": 28,
    "height": "6-1",
    "weight": "190 lbs.",
    "birthday": "January 17, 1991",
    "city": "North Hollywood",
    "number": 47,
    "position": "SP",
    "exp": "7 years",
    "team": "Cleveland Indians",
    "tid": "CLE",
    "throw":

{
    "pid": "35013",
    "fname": "Jake",
    "lname": "Bauers",
    "age": 23,
    "height": "6-1",
    "weight": "195 lbs.",
    "birthday": "October 6, 1995",
    "city": "Huntington Beach",
    "number": 10,
    "position": "LF",
    "exp": "1 year",
    "team": "Cleveland Indians",
    "tid": "CLE",
    "throw": "L",
    "bat": "L",
    "college": "None"
} 

{
    "pid": "41996",
    "fname": "Steven",
    "lname": "Kwan",
    "age": 0,
    "height": null,
    "weight": null,
    "birthday": null,
    "city": null,
    "number": -1,
    "position": null,
    "exp": null,
    "team": null,
    "tid": "CLE",
    "throw": null,
    "bat": null,
    "college": null
} 

{
    "pid": "37580",
    "fname": "Jordan",
    "lname": "Luplow",
    "age": 25,
    "height": "6-1",
    "weight": "195 lbs.",
    "birthday": "September 26, 1993",
    "city": "Visalia",
    "number": 8,
    "position": "RF",
    "exp": "2 years",
    "team": "Cleveland Indians",
    "tid": "CLE",
    "throw": "R",

{
    "pid": "32524",
    "fname": "Niko",
    "lname": "Goodrum",
    "age": 27,
    "height": "6-3",
    "weight": "198 lbs.",
    "birthday": "February 28, 1992",
    "city": "Fayetteville",
    "number": 28,
    "position": "SS",
    "exp": "2 years",
    "team": "Detroit Tigers",
    "tid": "DET",
    "throw": "B",
    "bat": "R",
    "college": "None"
} 

{
    "pid": "33195",
    "fname": "Dawel",
    "lname": "Lugo",
    "age": 24,
    "height": "6-0",
    "weight": "190 lbs.",
    "birthday": "December 31, 1994",
    "city": "Bani",
    "number": 18,
    "position": "2B",
    "exp": "1 year",
    "team": "Detroit Tigers",
    "tid": "DET",
    "throw": "R",
    "bat": "R",
    "college": "None"
} 

{
    "pid": "32131",
    "fname": "Ronny",
    "lname": "Rodriguez",
    "age": 27,
    "height": "6-0",
    "weight": "170 lbs.",
    "birthday": "April 17, 1992",
    "city": "Santiago",
    "number": 60,
    "position": "3B",
    "exp": "1 year",
    "team": "Detroit Tigers",
  

{
    "pid": "31682",
    "fname": "Cheslor",
    "lname": "Cuthbert",
    "age": 26,
    "height": "6-1",
    "weight": "190 lbs.",
    "birthday": "November 16, 1992",
    "city": "Bluefields",
    "number": 19,
    "position": "3B",
    "exp": "4 years",
    "team": "Kansas City Royals",
    "tid": "KC",
    "throw": "R",
    "bat": "R",
    "college": "None"
} 

{
    "pid": "30953",
    "fname": "Lucas",
    "lname": "Duda",
    "age": 33,
    "height": "6-4",
    "weight": "255 lbs.",
    "birthday": "February 3, 1986",
    "city": "Riverside",
    "number": 9,
    "position": "1B",
    "exp": "9 years",
    "team": "Kansas City Royals",
    "tid": "KC",
    "throw": "L",
    "bat": "R",
    "college": "USC"
} 

{
    "pid": "38106",
    "fname": "Nicky",
    "lname": "Lopez",
    "age": 0,
    "height": null,
    "weight": null,
    "birthday": null,
    "city": null,
    "number": -1,
    "position": null,
    "exp": null,
    "team": null,
    "tid": "KC",
    "throw": null,
 

...

{
    "pid": "32512",
    "fname": "Ketel",
    "lname": "Marte",
    "age": 25,
    "height": "6-1",
    "weight": "165 lbs.",
    "birthday": "October 12, 1993",
    "city": "Nizao",
    "number": 4,
    "position": "CF",
    "exp": "4 years",
    "team": "Arizona Diamondbacks",
    "tid": "ARI",
    "throw": "B",
    "bat": "R",
    "college": "None"
} 

{
    "pid": "33384",
    "fname": "David",
    "lname": "Peralta",
    "age": 31,
    "height": "6-1",
    "weight": "210 lbs.",
    "birthday": "August 14, 1987",
    "city": "Valencia",
    "number": 6,
    "position": "RF",
    "exp": "5 years",
    "team": "Arizona Diamondbacks",
    "tid": "ARI",
    "throw": "L",
    "bat": "L",
    "college": "None"
} 

{
    "pid": "31718",
    "fname": "Chad",
    "lname": "Bettis",
    "age": 30,
    "height": "6-1",
    "weight": "200 lbs.",
    "birthday": "April 26, 1989",
    "city": "Lubbock",
    "number": 35,
    "position": "SP",
    "exp": "6 years",
    "team": "Colorado Rockies

{
    "pid": "33364",
    "fname": "Pat",
    "lname": "Valaika",
    "age": 26,
    "height": "5-11",
    "weight": "200 lbs.",
    "birthday": "September 9, 1992",
    "city": "Valencia",
    "number": 4,
    "position": "3B",
    "exp": "3 years",
    "team": "Colorado Rockies",
    "tid": "COL",
    "throw": "R",
    "bat": "R",
    "college": "UCLA"
} 

{
    "pid": "31084",
    "fname": "Charlie",
    "lname": "Blackmon",
    "age": 32,
    "height": "6-3",
    "weight": "210 lbs.",
    "birthday": "July 1, 1986",
    "city": "Dallas",
    "number": 19,
    "position": "CF",
    "exp": "8 years",
    "team": "Colorado Rockies",
    "tid": "COL",
    "throw": "L",
    "bat": "L",
    "college": "Georgia Tech"
} 

{
    "pid": "32698",
    "fname": "David",
    "lname": "Dahl",
    "age": 25,
    "height": "6-2",
    "weight": "195 lbs.",
    "birthday": "April 1, 1994",
    "city": "Birmingham",
    "number": 26,
    "position": "LF",
    "exp": "2 years",
    "team": "Colorado Ro

{
    "pid": "29607",
    "fname": "Justin",
    "lname": "Turner",
    "age": 34,
    "height": "5-11",
    "weight": "205 lbs.",
    "birthday": "November 23, 1984",
    "city": "Long Beach",
    "number": 10,
    "position": "3B",
    "exp": "10 years",
    "team": "Los Angeles Dodgers",
    "tid": "LAD",
    "throw": "R",
    "bat": "R",
    "college": "Cal State Fullerton"
} 

{
    "pid": "35431",
    "fname": "Kyle",
    "lname": "Garlick",
    "age": 0,
    "height": null,
    "weight": null,
    "birthday": null,
    "city": null,
    "number": -1,
    "position": null,
    "exp": null,
    "team": null,
    "tid": "LAD",
    "throw": null,
    "bat": null,
    "college": null
} 

{
    "pid": "31358",
    "fname": "Enrique",
    "lname": "Hernandez",
    "age": 27,
    "height": "5-11",
    "weight": "200 lbs.",
    "birthday": "August 24, 1991",
    "city": "San Juan",
    "number": 14,
    "position": "CF",
    "exp": "5 years",
    "team": "Los Angeles Dodgers",
    "tid":

{
    "pid": "6197",
    "fname": "Ian",
    "lname": "Kinsler",
    "age": 36,
    "height": "6-0",
    "weight": "200 lbs.",
    "birthday": "June 22, 1982",
    "city": "Tucson",
    "number": 3,
    "position": "2B",
    "exp": "13 years",
    "team": "San Diego Padres",
    "tid": "SD",
    "throw": "R",
    "bat": "R",
    "college": "Missouri"
} 

{
    "pid": "31097",
    "fname": "Manny",
    "lname": "Machado",
    "age": 26,
    "height": "6-3",
    "weight": "215 lbs.",
    "birthday": "July 6, 1992",
    "city": "Miami",
    "number": 13,
    "position": "SS",
    "exp": "7 years",
    "team": "San Diego Padres",
    "tid": "SD",
    "throw": "R",
    "bat": "R",
    "college": "None"
} 

{
    "pid": "35983",
    "fname": "Fernando",
    "lname": "Tatis",
    "age": 0,
    "height": null,
    "weight": null,
    "birthday": null,
    "city": null,
    "number": -1,
    "position": null,
    "exp": null,
    "team": null,
    "tid": "SD",
    "throw": null,
    "bat": null

{
    "pid": "31032",
    "fname": "Stephen",
    "lname": "Vogt",
    "age": 34,
    "height": "6-0",
    "weight": "225 lbs.",
    "birthday": "November 1, 1984",
    "city": "Visalia",
    "number": 21,
    "position": "C",
    "exp": "6 years",
    "team": "San Francisco Giants",
    "tid": "SF",
    "throw": "L",
    "bat": "R",
    "college": "Azusa Pacific"
} 

{
    "pid": "32161",
    "fname": "Tyler",
    "lname": "Austin",
    "age": 27,
    "height": "6-1",
    "weight": "220 lbs.",
    "birthday": "September 6, 1991",
    "city": "Conyers",
    "number": 19,
    "position": "1B",
    "exp": "3 years",
    "team": "San Francisco Giants",
    "tid": "SF",
    "throw": "R",
    "bat": "R",
    "college": "None"
} 

{
    "pid": "30901",
    "fname": "Brandon",
    "lname": "Belt",
    "age": 31,
    "height": "6-4",
    "weight": "235 lbs.",
    "birthday": "April 20, 1988",
    "city": "Nacogdoches",
    "number": 9,
    "position": "1B",
    "exp": "8 years",
    "team": "S

In [ ]:
# Convert <Dictionary> 'mlb_player_data' to DataFrame
col_order = [
    'pid',
    'fname',
    'lname',
    'age',
    'height',
    'weight',
    'birthday',
    'city',
    'number',
    'position',
    'exp',
    'team',
    'throw',
    'bat',
    'college'
]
mlb_players_df = pd.DataFrame(mlb_player_data)[col_order]
mlb_players_df.head()

In [ ]:
# Export MLB Player DataFrame to CSV w/ Ordered Columns
mlb_players_df.to_csv(r'2019-MLB_Player_Data.csv', index=False)